In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3#,4,5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3#,4,5


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
from ganime.data.base import load_dataset
from ganime.model.vq_vae.vq_vae import VQVAE
from ganime.visualization.videos import display_videos
import tensorflow as tf
from tqdm import tqdm
tf.get_logger().setLevel('ERROR')

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
train_ds, test_ds, input_shape = load_dataset("moving_mnist_vae", "../../../data", batch_size=128)

NameError: name 'load_dataset' is not defined

In [15]:
strategy = tf.distribute.MirroredStrategy()

In [16]:
import tensorflow_datasets as tfds
def ds_stats(ds):
    ds_numpy = tfds.as_numpy(ds)
    data = []
    
    for da in tqdm(ds_numpy):
        X, y = da
        data.append(X)
    all_data = np.concatenate(data)
    return np.mean(all_data), np.var(all_data), np.std(all_data)

In [17]:
train_mean, train_var, train_std = ds_stats(train_ds)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71/71 [00:01<00:00, 40.04it/s]


In [22]:
with strategy.scope():
    model = VQVAE(train_variance=train_var)

In [23]:
model.encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 20, 64, 64, 1)]   0         
                                                                 
 time_distributed_18 (TimeDi  (None, 20, 32, 32, 32)   320       
 stributed)                                                      
                                                                 
 time_distributed_19 (TimeDi  (None, 20, 16, 16, 64)   18496     
 stributed)                                                      
                                                                 
 time_distributed_20 (TimeDi  (None, 20, 16, 16, 32)   2080      
 stributed)                                                      
                                                                 
Total params: 20,896
Trainable params: 20,896
Non-trainable params: 0
_______________________________________________________

In [24]:
model.decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 20, 16, 16, 32)]  0         
                                                                 
 time_distributed_24 (TimeDi  (None, 20, 32, 32, 64)   18496     
 stributed)                                                      
                                                                 
 time_distributed_25 (TimeDi  (None, 20, 64, 64, 32)   18464     
 stributed)                                                      
                                                                 
 time_distributed_26 (TimeDi  (None, 20, 64, 64, 1)    289       
 stributed)                                                      
                                                                 
Total params: 37,249
Trainable params: 37,249
Non-trainable params: 0
_______________________________________________________

In [25]:
model.vqvae.summary()

Model: "vq_vae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 20, 64, 64, 1)]   0         
                                                                 
 encoder (Functional)        (None, 20, 16, 16, 32)    20896     
                                                                 
 vector_quantizer (VectorQua  (None, 20, 16, 16, 32)   4096      
 ntizer)                                                         
                                                                 
 decoder (Functional)        (None, 20, 64, 64, 1)     37249     
                                                                 
Total params: 62,241
Trainable params: 62,241
Non-trainable params: 0
_________________________________________________________________


In [26]:
with strategy.scope():
    model.compile(optimizer=tf.keras.optimizers.Adam())
history = model.fit(train_ds, epochs=100)

2022-05-05 07:05:10.846606: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:766] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_1"
op: "TensorSliceDataset"
input: "Placeholder/_0"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_UINT8
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 9000
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\025TensorSliceDataset:10"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 20
        }
        dim {
          size: 64
        }
        dim {
          size: 64
        }
        dim {
          size: 1
        }
      }
    }
  }
}



Epoch 1/100


2022-05-05 07:05:22.859011: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-05 07:05:23.626380: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-05 07:05:24.695855: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-05 07:05:25.882980: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-05 07:05:26.584680: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-05 07:05:26.647228: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-05-05 07:05:27.819886: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-05 07:05:28.894294: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8101
2022-05-05 07:05:29.750438: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN ve

71/71 [==============================] - 26s 67ms/step - loss: 3.4236 - reconstruction_loss: 0.7678 - vqvae_loss: 2.5013
Epoch 2/100
71/71 [==============================] - 4s 53ms/step - loss: 217.1925 - reconstruction_loss: 0.4496 - vqvae_loss: 216.7030
Epoch 3/100
71/71 [==============================] - 4s 55ms/step - loss: 65.5686 - reconstruction_loss: 0.2874 - vqvae_loss: 65.2724
Epoch 4/100
71/71 [==============================] - 4s 57ms/step - loss: 29.4214 - reconstruction_loss: 0.2824 - vqvae_loss: 29.1413
Epoch 5/100
71/71 [==============================] - 4s 53ms/step - loss: 24.2109 - reconstruction_loss: 0.2868 - vqvae_loss: 23.9238
Epoch 6/100
71/71 [==============================] - 4s 53ms/step - loss: 19.6524 - reconstruction_loss: 0.2583 - vqvae_loss: 19.3877
Epoch 7/100
71/71 [==============================] - 4s 57ms/step - loss: 20.9975 - reconstruction_loss: 0.2898 - vqvae_loss: 20.7194
Epoch 8/100
71/71 [==============================] - 4s 54ms/step - loss:

In [43]:
train_ds_iterator = train_ds.as_numpy_iterator()
x = next(train_ds_iterator)

In [44]:
generated = model(x[0])

In [45]:
display_videos(generated, n_rows=1, n_cols=5)

In [46]:
display_videos(x[1], n_rows=1, n_cols=5)

In [39]:
test_ds_iterator = test_ds.as_numpy_iterator()
x = next(test_ds_iterator)

In [40]:
generated = model(x[0])

In [41]:
display_videos(generated, n_rows=1, n_cols=5)

In [42]:
display_videos(x[1], n_rows=1, n_cols=5)